In [0]:
import pandas as pd
import numpy as np

column_names = ['user_id', 'item_id', 'rating', 'timestamp']
dataframe = pd.read_csv('u.data', sep='\t', names= column_names, encoding='latin-1')


In [3]:
Data_NULL = dataframe.isnull().sum()*100/dataframe.shape[0]
Data_NULL

user_id      0.0
item_id      0.0
rating       0.0
timestamp    0.0
dtype: float64

No need to fill missing values as no NaN exist

In [0]:
#dataframe['user_id'].fillna(0, inplace=True)
#dataframe['item_id'].fillna(0, inplace=True)
#dataframe['rating'].fillna(dataframe['rating'].mean(), inplace=True)

In [5]:
Data_NULL = dataframe.isnull().sum()*100/dataframe.shape[0]
Data_NULL

user_id      0.0
item_id      0.0
rating       0.0
timestamp    0.0
dtype: float64

In [6]:
dataframe.describe()

,user_id,item_id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [7]:
dataframe

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [0]:
users = dataframe.user_id.unique().shape[0]
items = dataframe.item_id.unique().shape[0]

all_ratings = np.zeros((users, items))
for r in dataframe.itertuples():
    all_ratings[r[1]-1, r[2]-1] = r[3]

In [9]:
all_ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

Functions to perform prediction

In [0]:
import numpy as np
def perform_userwise_predition(all_ratings, user_similarity, k, item, user):
    topk = user_similarity.argsort()[-k:][::-1]
    user = user -1
    item = item - 1
    curruserData = np.delete(all_ratings, item, axis=1)
    #getting current users average rating
    avguserRating = np.mean(curruserData[user,:])
    totalr = 0
    my_list = []
    #iterating over all neighbours to get get the similarity * (their rating - mean)
    for i in topk:
        if all_ratings[i][item] != 0:
            totalr += user_similarity[i] * (all_ratings[i][item] - (np.mean(all_ratings[i,:])))
            my_list.append(user_similarity[i])
    #divide by similarities
    if len(my_list) > 0:
        totalr = totalr / np.sum(my_list)
    #return rating + curruser Average
    return (totalr + avguserRating)

In [0]:
def perform_itemwise_predition(all_ratings, item_similarity, k, item, user):
    topk = item_similarity.argsort()[-k:][::-1]
    user = user -1
    item = item - 1
    #taking transpose of matrix so same logic as user similarity can be reused
    all_ratings =all_ratings.transpose()
    #get current item average rating
    currItemData = np.delete(all_ratings, user, axis=1)
    avguItemRating = np.mean(currItemData[item,:])
    totalr = 0
    my_list = []
    #iterating over top k neighbours to get the numerator for the equation
    for i in topk:
        if all_ratings[i][user] != 0:
            totalr += item_similarity[i] * (all_ratings[i][user] - (np.mean(all_ratings[i,:])))
            my_list.append(item_similarity[i])
    #divide by similarities
    if len(my_list) > 0:
        totalr = totalr / np.sum(my_list)
    #return average plus calculated rating
    return (totalr + avguItemRating)

Functions to get similarity in between users or items

In [0]:
import math 
def get_user_similarity(all_ratings,userid,item):
    userid = userid-1
    item = item - 1
    all_ratingscopy = np.delete(all_ratings, item, axis=1)
    arr=np.zeros(all_ratingscopy.shape[0])
    for i in range(0,all_ratingscopy.shape[0]):
        if i==userid:
            arr[i] = 0
        else:
            sum = 0
            for j in range(0, all_ratingscopy.shape[1]):
                sum += all_ratingscopy[userid][j] * all_ratingscopy[i][j]
            #handle division by zero case
            if (np.sum(np.square(all_ratingscopy[i])) == 0):
                arr[i] = 0
            else:
                arr[i] = sum/(math.sqrt(np.sum(np.square(all_ratingscopy[userid]))) * math.sqrt(np.sum(np.square(all_ratingscopy[i]))))
    return arr

In [0]:
def get_item_similarity(all_ratings,userid,item):
    userid = userid-1
    item = item - 1
    #taking transpose of matrix so same logic as user similarity can be reused
    all_ratings = all_ratings.transpose()
    all_ratingscopy = np.delete(all_ratings, userid, axis=1)
    arr=np.zeros(all_ratingscopy.shape[0])
    for i in range(0,all_ratingscopy.shape[0]):
        if i==item:
            arr[i] = 0
        else:
            sum = 0
            for j in range(0, all_ratingscopy.shape[1]):
                sum += all_ratingscopy[item][j] * all_ratingscopy[i][j]
            #print(i)
            #handle division by zero case
            if (np.sum(np.square(all_ratingscopy[i])) == 0):
                arr[i] = 0
            else :
                arr[i] = sum/(math.sqrt(np.sum(np.square(all_ratingscopy[item]))) * math.sqrt(np.sum(np.square(all_ratingscopy[i]))))
    return arr

In [0]:
def predict():
  user_id = int(input("Please enter the user id :"))
  item_id = int(input("Please enter the item id :"))
  k = int(input("Please input the neighbourhood size :"))

  row = dataframe[(dataframe['user_id']==user_id) & (dataframe['item_id']==item_id)]
  index = row.index

  itemwise_get_similarity = get_item_similarity(all_ratings,user_id,item_id)
  userwise_get_similarity = get_user_similarity(all_ratings,user_id,item_id)

  item_prediction = perform_itemwise_predition(all_ratings, itemwise_get_similarity, k,item_id,user_id)
  user_prediction = perform_userwise_predition(all_ratings, userwise_get_similarity, k,item_id,user_id)

  print("Item-based Collaborative Filtering Prediction:", item_prediction)
  print("User-based Collaborative Filtering Prediction:", user_prediction)

In [15]:
predict()

Please enter the user id :198
Please enter the item id :96
Please input the neighbourhood size :3
Item-based Collaborative Filtering Prediction: 3.6344695935240856
User-based Collaborative Filtering Prediction: 4.166122479657569


In [16]:
predict()

Please enter the user id :210
Please enter the item id :222
Please input the neighbourhood size :10
Item-based Collaborative Filtering Prediction: 4.197497268850702
User-based Collaborative Filtering Prediction: 3.1488845586663157
